In [3]:
!pip3 install tqdm
from tqdm import tqdm
import pandas as pd
import numpy as np
import json

    100% |████████████████████████████████| 51kB 1.2MB/s ta 0:00:01


In [45]:
raw_data = []
cpi = {}

with open('raw_data.json', 'r') as infile:  
    raw_data = pd.read_json(infile, orient='records')
    
with open('cpi.json', 'r') as cpi_file:
    cpi = eval(cpi_file.read())

def adjust_for_inflation(amount, year):
    current = cpi[str(2017)]
    past = cpi[str(year)]
    return int((current/past) * amount)
    
raw_data.head()

,box_office,budget,cast,release_date,title,url
0,1653792.0,NaN,"[Chris Rock, Julie Delpy, Albert Delpy, Alexia...",2012-03-28,2 Days in New York,/wiki/2_Days_in_New_York
1,675000.0,NaN,"[Anton Yelchin, Bérénice Marlohe, Olivia Thirl...",2014-04-19,5 to 7,/wiki/5_to_7
2,1370000.0,607000.0,"[Ginger Rogers, Walter Connolly]",1939-09-22,5th Ave Girl,/wiki/5th_Ave_Girl
3,74800000.0,7000000.0,"[Dudley Moore, Julie Andrews, Robert Webber, B...",1979-10-05,10,/wiki/10_(film)
4,NaN,NaN,"[Jesse Bradford, Chris Marquette, Tammin Surso...",2013-08-13,10 Rules for Sleeping Around,/wiki/10_Rules_for_Sleeping_Around


In [7]:
raw_data["release_date"] = pd.to_datetime(raw_data["release_date"])

casts = raw_data[raw_data["cast"].astype(str) != "[]"]
dates = raw_data[pd.notnull(raw_data["release_date"])]
money = raw_data[pd.notnull(raw_data["box_office"])]
money = money[pd.notnull(money["budget"])]
cleaned = raw_data.dropna()


cleaned.head()

,box_office,budget,cast,release_date,title,url
2,1370000.0,607000.0,"[Ginger Rogers, Walter Connolly]",1939-09-22,5th Ave Girl,/wiki/5th_Ave_Girl
3,74800000.0,7000000.0,"[Dudley Moore, Julie Andrews, Robert Webber, B...",1979-10-05,10,/wiki/10_(film)
8,96500000.0,37000000.0,"[Jennifer Garner, Mark Ruffalo, Judy Greer, An...",2004-04-23,13 Going on 30,/wiki/13_Going_on_30
10,136300000.0,40000000.0,"[Zac Efron, Leslie Mann, Thomas Lennon, Michel...",2009-04-17,17 Again,/wiki/17_Again_(film)
11,160300000.0,30000000.0,"[Katherine Heigl, James Marsden, Malin Åkerman...",2008-01-10,27 Dresses,/wiki/27_Dresses


In [8]:
print("""
raw  : {}
casts: {}
dates: {}
money: {}
clean: {}
""".format(len(raw_data), len(casts), len(dates), len(money), len(cleaned)))


raw  : 1740
casts: 1692
dates: 1568
money: 643
clean: 629



In [9]:
actors_map = {}

for i, movie in casts.iterrows():
    cast = movie["cast"]
    for actor_name in cast:
        actor = actors_map.get(actor_name, {"links": {}, "movies": [], "total": 0})
        actor["movies"].append(movie["title"])
        movie_index = len(actor["movies"])-1
        for other_name in cast:
            if actor_name == other_name:
                continue
            shared = actor["links"].get(other_name, [])
            shared.append(movie_index)
            actor["links"][other_name] = shared
            actor["total"] = actor["total"] + 1
        actors_map[actor_name] = actor

In [10]:
print("Links\tNodes")
print("{}-4:\t{}".format(1, len([actors_map[actor] for actor in actors_map.keys() if actors_map[actor]["total"] >= 1])))
for i in range(5, 81, 5):
    print("{}-{}:\t{}".format(i, i+4, len([actors_map[actor] for actor in actors_map.keys() if i+5 > actors_map[actor]["total"] >= i])))


Links	Nodes
1-4:	3996
5-9:	1614
10-14:	521
15-19:	264
20-24:	119
25-29:	93
30-34:	43
35-39:	16
40-44:	16
45-49:	10
50-54:	7
55-59:	7
60-64:	3
65-69:	3
70-74:	0
75-79:	1
80-84:	0


In [12]:
sorted_dates = dates[["title","release_date"]].sort_values("release_date")

def comparator(movie):
    filtered = sorted_dates[sorted_dates["title"] == movie]
    if not filtered.empty:
        index = sorted_dates.index.get_loc(filtered.iloc[0].name)
    else:
        return -250
    return index

def get_year(title):
    try:
        row = sorted_dates[sorted_dates["title"] == title].iloc[0]
        median = row["release_date"].year
        return median
    except IndexError:
        return -1

for actor_name in tqdm(actors_map):
    actor = actors_map[actor_name]
    if len(actor["movies"]) > 1:
        movie_list = sorted(actor["movies"].copy(), key=comparator)
        median_index = (len(movie_list)+1)//2
        median = get_year(movie_list[median_index])
    else:
        median = get_year(actor["movies"][0])
    actor["median_year"] = median
    
    

100%|██████████| 4010/4010 [00:07<00:00, 540.31it/s]


In [216]:
i = 0
for actor_name in actors_map:
    actor = actors_map[actor_name]
    if actor["median_year"] == -1:
        i += 1
        
i

276

# Todo:
* Good luck, you might have to add in median years fo 276 actors manually...

In [239]:
nodes = []
links = []
for actor_name in actors_map:
    actor = actors_map[actor_name]
    nodes.append({
        "name": actor_name,
        "total": actor["total"],
        "movies": actor["movies"],
        "median": actor["median_year"]
    })
    for link in actor["links"]:
        links.append({
            "source": actor_name,
            "target": link,
            "value": actor["links"][link]
        })

In [247]:
print("Weight\tLinks")
for i in range(1,11):
    print("{}:\t{}".format(i,len([link for link in links if len(link["value"]) >= i])))

Weight	Links
1:	33656
2:	528
3:	74
4:	16
5:	8
6:	6
7:	2
8:	0
9:	0
10:	0


528

In [263]:
new_links = [link for link in links if len(link["value"]) > 1]
hashed_links = {link["source"]: True for link in new_links}

new_nodes = [node for node in nodes if hashed_links.get(node["name"], False)]

In [271]:
with open('network.json', 'w') as outfile:  
    network_data = {
        "nodes": new_nodes,
        "links": new_links
    }
    json.dump(network_data, outfile, ensure_ascii=False, indent=4)

In [33]:
scatter_data = [{
                    "box_office": movie["box_office"],
                    "budget": movie["budget"],
                    "year": get_year(movie["title"]),
                    "title": movie["title"]
                } for i, movie in money.iterrows()]

with open('scatter.json', 'w') as outfile:  
    json.dump(scatter_data, outfile, ensure_ascii=False, indent=4)

In [313]:
# sorted_dates.groupby((sorted_dates.set_index("release_date").index.year//10)*10).apply(lambda x: x.to_json())
bins = {}
for i, movie in list(sorted_dates.iterrows())[:]:
    decade = (movie["release_date"].year//10)*10
    decade_bin = bins.get(decade, [])
    decade_bin.append(movie["title"])
    bins[decade] = decade_bin

In [316]:
for decade in bins:
    print("{}: {}".format(decade, len(bins[decade])))

1910: 37
1920: 90
1930: 230
1940: 190
1950: 101
1960: 81
1970: 36
1980: 109
1990: 192
2000: 327
2010: 175


In [317]:
with open('decades.json', 'w') as outfile:  
    json.dump(bins, outfile, ensure_ascii=False, indent=4)

In [324]:
actor_bins = {}

for actor_name in actors_map:
    actor = actors_map[actor_name]
    decade = actor["median_year"]
    if decade == -1:
        continue
    decade = (decade//10) * 10
    decade_bin = actor_bins.get(decade, [])
    decade_bin.append(actor_name)
    actor_bins[decade] = decade_bin
    
with open('actor-decades.json', 'w') as outfile:  
    json.dump(actor_bins, outfile, ensure_ascii=False, indent=4)

In [326]:
for decade in sorted(actor_bins.keys()):
    print("{}: {}".format(decade, len(actor_bins[decade])))

1910: 36
1920: 111
1930: 302
1940: 239
1950: 180
1960: 174
1970: 75
1980: 247
1990: 546
2000: 1085
2010: 739


# After prototyping, I went back to modify my output data to feature new fields that I figured out that I had needed and also computed other fields to optimize it now before bringing it into the JS

In [62]:
# with open('scatter.json', 'r') as infile:  
#     scatter_data = pd.read_json(infile)

scatter_data["budget"] = scatter_data["budget"].astype("int")
scatter_data["box_office"] = scatter_data["box_office"].astype("int")
scatter_data["budget_adj"] = np.vectorize(adjust_for_inflation)(scatter_data["budget"], scatter_data["year"])
scatter_data["box_office_adj"] = np.vectorize(adjust_for_inflation)(scatter_data["box_office"], scatter_data["year"])


with open('scatter.json', 'w') as outfile:
    out_json = json.loads(scatter_data.to_json(orient='records'))
    json.dump(out_json, outfile, ensure_ascii=False, indent=4)

In [70]:
with open('decades.json', 'r') as decades_file:  
    decades_data = pd.Series(eval(decades_file.read()), name="movies").to_frame()

decades_data["length"] = np.vectorize(len)(decades_data)
out_json = json.loads(decades_data.to_json(orient='index'))

with open('decades.json', 'w') as outfile:
    json.dump(out_json, outfile, ensure_ascii=False, indent=4)

In [76]:
with open('actor-decades.json', 'r') as actors_file:  
    actors_data = pd.Series(eval(actors_file.read()), name="actors").to_frame()

actors_data["length"] = np.vectorize(len)(actors_data)
out_json = json.loads(actors_data.to_json(orient='index'))

with open('actor-decades.json', 'w') as outfile:
    json.dump(out_json, outfile, ensure_ascii=False, indent=4)

In [91]:
with open('network.json', 'r') as network_file:  
    network = eval(network_file.read())
    nodes = network["nodes"]
    links = network["links"]

for node in nodes:
    node["movie_total"] = len(node["movies"])
out_json = {"nodes": nodes, "links": links}

with open('network.json', 'w') as outfile:
    json.dump(out_json, outfile, ensure_ascii=False, indent=4)

In [8]:
with open('scatter.json', 'r') as scatter_file:
    scatter_data = pd.Series(eval(scatter_file.read()), name="points").to_frame()
    
scatter_data.head()

,points
1920,"[{'box_office': 1550000, 'box_office_adj': 221..."
1930,"[{'box_office': 1370000, 'box_office_adj': 240..."
1940,"[{'box_office': 3947000, 'box_office_adj': 405..."
1950,"[{'box_office': 3475000, 'box_office_adj': 290..."
1960,"[{'box_office': 24600000, 'box_office_adj': 20..."


In [21]:
decades_indices = sorted(set(scatter_data["decade"]))
new_scatter = {}
for decade in decades_indices:
    subset = scatter_data[scatter_data["decade"] == decade]
    new_scatter[str(decade)] = json.loads(subset.to_json(orient='records'))

with open('scatter.json', 'w') as outfile:
    json.dump(new_scatter, outfile, ensure_ascii=False, indent=4)

In [28]:
with open('actor-decades.json', 'r') as actors_file:  
    actors_data = pd.read_json(actors_file, orient="index")
    
with open('decades.json', 'r') as decades_file:  
    decades_data = pd.read_json(decades_file, orient="index")

ratios = (actors_data["length"]/decades_data["length"]).round(2)

ratio_data = pd.DataFrame(data=ratios, index = actors_data.index)

with open('ratios.json', 'w') as outfile:
    out_json = json.loads(ratio_data.to_json(orient='index'))
    json.dump(out_json, outfile, ensure_ascii=False, indent=4)